<a href="https://colab.research.google.com/github/polina-minaeva/Hybrid-recommender-systems/blob/main/5_%D0%93%D0%B8%D0%B1%D1%80%D0%B8%D0%B4%D0%BD%D1%8B%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Построим гибридную рекомендательную систему на основе модели из Surprise и векторов tfidf

### 1. Загрузка данных

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162986 sha256=a026ab1cca891df26ee5c73c26819340839387ecd9608afed4c97b5236856b07
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import KNNBasic, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-04-14 13:59:54--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 142.251.180.139, 142.251.180.101, 142.251.180.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.180.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-04-14 13:59:54--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.148.132, 2607:f8b0:4001:c0b::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.148.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.009s  

2024-04-14 13:59:55 (88.5 M

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
movies = pd.read_csv('movies.csv') #информация о фильмах
ratings = pd.read_csv('ratings.csv') #рейтинги
links = pd.read_csv('links.csv')
tags = pd.read_csv('tags.csv') #теги

In [ ]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

### 3. Построение модели коллаборативной фильтрации

In [ ]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [ ]:
%%time
algo = KNNBasic(k=50, min_k=2) #пропускаем данные через модель KNNBasic
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
CPU times: user 146 ms, sys: 8.58 ms, total: 155 ms
Wall time: 156 ms


In [ ]:
test_pred = algo.test(testset)

In [ ]:
accuracy.rmse(test_pred, verbose=True) #оценка качечства модели

RMSE: 0.9401


0.9401041805114329

In [ ]:
algo.predict(uid=2.0, iid='Mortal Kombat (1995)').est #проверяем модель, какой рейтинг имеет этот фильм

2.472837148143217

In [ ]:
current_user_id = 2.0
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_with_ratings.title.unique():
    if movie in user_movies: #исключим фильмы, которые юзер уже смотрел, - их не будем рекомендовать
        continue

    scores.append(algo.predict(uid=current_user_id, iid=movie).est) # у фильмов добавляем в список их предполагаемую оценку
    titles.append(movie)

In [ ]:
scores

[3.876435550778097,
 3.38082775467615,
 3.3706779402560394,
 2.491957630443311,
 2.746547615678709,
 4.0782183734259,
 3.1224936216480796,
 2.1996724984917693,
 2.524945770065076,
 3.6255647875452044,
 3.6670537478979193,
 2.969639743333494,
 3.225606608156944,
 3.8293838862559233,
 3.157751523326667,
 4.07523345685849,
 3.8304260694111907,
 3.6981403041003844,
 2.871797687959114,
 2.459431910936975,
 3.538487417197367,
 3.2977649513662155,
 3.155263640819455,
 3.160645538452387,
 3.654270928267277,
 3.4470242565185125,
 3.578268267322053,
 4.109531064013825,
 3.9874128884731515,
 4.0,
 3.3151947441292826,
 4.016892425959146,
 3.587419695954789,
 3.7994705814243694,
 2.3298969072164954,
 3.5205776358003775,
 3.502998483257496,
 3.678400174367916,
 2.9633037669490587,
 2.8546276853497012,
 2.472837148143217,
 3.3556129983828096,
 3.528690232593066,
 3.9157888669126817,
 3.0648768937672273,
 3.502998483257496,
 4.196106730445261,
 3.448605316030053,
 5.0,
 2.155172413793103,
 3.502998483

In [ ]:
sorted(scores)[-10:] #можно отсортировать фильмы с лучшим рейтингом

[4.847058823529411,
 4.851393188854489,
 4.858634742740703,
 4.863636363636364,
 4.8724391186702745,
 4.890952666267225,
 5.0,
 5.0,
 5.0,
 5.0]

### 4. Получение векторов tfidf

In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [ ]:
movie_genres = [change_string(g) for g in movies.genres.values] #вектора будем строить по жанрам фильмов

In [ ]:
movie_genres[0]

'Adventure Animation Children Comedy Fantasy'

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean') #будем рекомендовать 20 подходящих фильмов
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=20)

In [ ]:
test = change_string("Adventure|Comedy|Fantasy|Crime") #протестируем алгоритм, подберем к фильму с такими жанрами - подходящие фильмы

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [ ]:
res #нам выдали коэффициенты похожести и индекс подходящих фильмов

(array([[0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.6188388 , 0.62682864, 0.62682864]]),
 array([[6774, 9096, 2302, 5627, 6723, 8361, 5737, 2608, 7865, 5636, 3302,
         7496, 3582,  863, 3376, 9717, 3576, 2206, 3010, 6133]]))

In [ ]:
movies.iloc[res[1][0]] #смотрим по индексам, какие фильмы нам порекомендовали

,movieId,title,genres
6774,60074,Hancock (2008),Action|Adventure|Comedy|Crime|Fantasy
9096,143559,L.A. Slasher (2015),Comedy|Crime|Fantasy
2302,3052,Dogma (1999),Adventure|Comedy|Fantasy
5627,27251,"10th Kingdom, The (2000)",Adventure|Comedy|Fantasy
6723,58972,Nim's Island (2008),Adventure|Comedy|Fantasy
8361,109042,Knights of Badassdom (2013),Adventure|Comedy|Fantasy
5737,30810,"Life Aquatic with Steve Zissou, The (2004)",Adventure|Comedy|Fantasy
2608,3489,Hook (1991),Adventure|Comedy|Fantasy
7865,94015,Mirror Mirror (2012),Adventure|Comedy|Fantasy
5636,27368,Asterix & Obelix: Mission Cleopatra (Astérix &...,Adventure|Comedy|Fantasy


In [ ]:
movies_with_ratings.sort_values('timestamp', inplace=True) #сортируем фильмы по времени просмотра

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
title_genres = {}

for index, row in tqdm_notebook(movies.iterrows()):
    title_genres[row.title] = row.genres

<ipython-input-30-bd73bb19affb>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm_notebook(movies.iterrows()):


0it [00:00, ?it/s]

### 5. Создание рекоммендаций

In [ ]:
def recommend_for_user(user_id):
    current_user_id = user_id #наш юзер
    user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique() #фильмы которые смотрел юзер

    last_user_movie = user_movies[-1] #какой фильм юзер посмотрел последним

    movie_genres = title_genres[last_user_movie] #жанры этого фильма

    movie_genres = change_string(movie_genres) #преобразуем строку для вектора tfidf

    predict = count_vect.transform([movie_genres])
    X_tfidf2 = tfidf_transformer.transform(predict) #наш вектор для жанров

    res = neigh.kneighbors(X_tfidf2, return_distance=True) #набор подходящих фильмов

    movies_to_score = movies.iloc[res[1][0]].title.values #достаем индексы этих фильмов

    scores = [] #рейтинги
    titles = []

    for movie in movies_to_score:
        if movie in user_movies: #не учитываем просмотренные фильмы
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est) #в список добавляем рейтинги для этих фильмов
        titles.append(movie)


    best_indexes = np.argsort(scores)[-10:] #отбираем лучшие
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [ ]:
movies_with_ratings[movies_with_ratings.userId == 2.0].sort_values('rating') #что смотрел юзер

,movieId,title,genres,userId,rating,timestamp
97478,114060,The Drop (2014),Crime|Drama|Thriller,2.0,2.0,1.445715e+09
93998,91658,"Girl with the Dragon Tattoo, The (2011)",Drama|Thriller,2.0,2.5,1.445715e+09
8652,318,"Shawshank Redemption, The (1994)",Crime|Drama,2.0,3.0,1.445715e+09
96746,109487,Interstellar (2014),Sci-Fi|IMAX,2.0,3.0,1.445715e+09
91063,77455,Exit Through the Gift Shop (2010),Comedy|Documentary,2.0,3.0,1.445715e+09
90135,71535,Zombieland (2009),Action|Comedy|Horror,2.0,3.0,1.445715e+09
93833,91529,"Dark Knight Rises, The (2012)",Action|Adventure|Crime|IMAX,2.0,3.5,1.445715e+09
95272,99114,Django Unchained (2012),Action|Drama|Western,2.0,3.5,1.445715e+09
97675,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2.0,3.5,1.445715e+09
76960,8798,Collateral (2004),Action|Crime|Drama|Thriller,2.0,3.5,1.445715e+09


In [ ]:
recommend_for_user(2.0) #что мы ему посоветовали

Taxi Driver (1976) 4.0662303625086595
Limey, The (1999) 3.9995552121615643
Shadow of a Doubt (1943) 3.975870703391759
Few Good Men, A (1992) 3.847663886230166
Cape Fear (1962) 3.8454752817324573
Simple Plan, A (1998) 3.8028349494856855
Rope (1948) 3.758121797343859
Jackie Brown (1997) 3.7069916350917347
Killing Zoe (1994) 3.6494377859626543
Fresh (1994) 3.544910179640719


In [ ]:
np.argsort([1,9,5,7])

array([0, 2, 3, 1])